In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

# Define paths based on extracted content
images_dir = 'D:/'
labels_excel = os.path.join(images_dir, '/Img/Book1.csv.xlsx')

# Function to load and preprocess images
def load_images_and_labels(images_directory, labels_excel_file, target_size=(28, 28)):
    images = []
    labels = []
    try:
        # Load the Excel file containing image names and labels
        labels_df = pd.read_excel(labels_excel_file)
        print(f"Excel file loaded successfully. Columns: {labels_df.columns}")
        
        # Iterate over the DataFrame to load images and corresponding labels
        for index, row in labels_df.iterrows():
            img_file = row['image'] + '.png'  # Add the correct file extension
            label = row['label']
            
            img_path = os.path.join(images_directory, img_file)
            if os.path.exists(img_path):
                # Load image and preprocess
                img = load_img(img_path, target_size=target_size, color_mode='grayscale')
                img_array = img_to_array(img)  # Keep the image in 3D array
                img_array = img_array / 255.0  # Normalize pixel values
                
                images.append(img_array)
                labels.append(label)
            else:
                print(f"Image file does not exist: {img_path}")
                
    except Exception as e:
        print(f"Error loading images or labels: {e}")
    
    return np.array(images), np.array(labels)

# Load images and labels
images, labels = load_images_and_labels(images_dir, labels_excel)

# Check if images are loaded
if images.size == 0:
    print("No images loaded. Check your directories and Excel file.")
else:
    # Shuffle the data
    images, labels = shuffle(images, labels, random_state=42)
    
    # Convert labels to categorical
    class_indices = {label: idx for idx, label in enumerate(np.unique(labels))}
    labels = np.array([class_indices[label] for label in labels])
    labels = to_categorical(labels)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    # Define the CNN model
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(class_indices), activation='softmax')  # adjust based on number of classes
    ])
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',  # use categorical crossentropy for one-hot encoded labels
                  metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=700, validation_split=0.2)
    
    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {test_acc * 100:.2f}%")
    
    # Print class labels
    print("Class Labels:")
    for label, idx in class_indices.items():
        print(f"{idx}: {label}")

    # Save the final model
    model.save('D:/character_recognition_model_cnn.h5')